### 阶段一数据集

In [5]:
import pandas as pd
import numpy as np
q_std_map = pd.read_csv('F:/Projects/data/corpus/qa/FinanceFAQ/input/q_std.tsv', sep='\t', encoding='utf-8')['0'].to_dict()
query_pair = pd.read_csv('F:/Projects/data/corpus/qa/FinanceFAQ/input/query_pair_0.tsv', sep='\t', encoding='utf-8')
query_pair['q_std'] = query_pair['q_std'].map(q_std_map)
query_pair.to_csv('F:/Projects/data/corpus/qa/FinanceFAQ/FinanceFAQ_train.tsv', sep='\t', encoding='utf-8', index=False)
query_pair.tail()

,q_std,q_sim
49995,评级家数小于#数字实体#家的#概念名称#基金有哪些,请把评级家数低于50家的HIT电池基金的清单给我念一下
49996,两融帐户首次激活后没有额度的原因,两融开户后没有授信额度
49997,国债逆回购是否计算持仓市值,听说国债逆回购是需要计算持仓市值的，这是真的吗
49998,系统维护期间是否可以正常委托,系统维护中可不可以开户吗
49999,人工服务电话和时间,客服电话来一个


In [6]:
query_pair.q_sim.str.len().describe()

count    50000.000000
mean        18.549940
std          7.961594
min          1.000000
25%         13.000000
50%         18.000000
75%         23.000000
max        108.000000
Name: q_sim, dtype: float64

In [9]:
# 为每个q_std随机选择一个q_sim作为dev集
q_std_list = query_pair['q_std'].unique().tolist()
query_pair['test_rnd'] = query_pair.q_std.apply(lambda x: np.random.rand())
query_pair['nrank_test'] = query_pair.groupby('q_std')['test_rnd'].rank(ascending=0, method='first')
dev_query_pair =  query_pair[query_pair.nrank_test<=1][['q_std', 'q_sim']]
dev_query_pair.head(5)

,q_std,q_sim
11,合约未了结情况下，卖出担保品时，提示零股不支持交易的解决方式,合约未了结情况下，卖出担保品时，解决提醒零股不能买卖的方式可能是什么
15,任职基金数大于#数字实体#的基金经理有哪些,想知道任职基金数大于50的基金经理有哪些
25,退市整理股票是否支持市价委托,问一下退市整理股票是否能做市价委托
31,竞价涨幅不小于#数字实体#的#地域板块#股票有哪些,给我发下都有啥内蒙板块股票是竞价涨幅不少于50
32,负债及持有人权益合计大于#数字实体#元的#概念名称#基金有哪些,都有什么中药基金是负债及持有人权益合计大于10元


In [10]:
# 为所有的query配一个qid
q_std_dev = dev_query_pair.q_std.unique().tolist()
q_sim_dev = dev_query_pair.q_sim.unique().tolist()
q_qid = q_std_dev + q_sim_dev
q_qid = list(set(q_qid))
q_qid_dict = {i+1:q_qid[i] for i in range(0, len(q_qid))}  # {id: query}
q_qid_dict_inv = {v: k for k, v in q_qid_dict.items()}  # {query: id}

# 建立ir_corpus： [q_id, q_std]的映射
ir_corpus = {q_qid_dict_inv[v]: v for v in q_std_list if v not in q_sim_dev}
ir_corpus_df = pd.DataFrame(list(ir_corpus.items()), columns=['qid', 'question']).sort_values('qid').reset_index(drop=True)
ir_corpus_df.to_csv('F:/Projects/data/corpus/qa/FinanceFAQ/ir_corpus.tsv', sep='\t', index=False)

# 保存dev
dev_query_pair['qid'] = dev_query_pair.q_sim.map(q_qid_dict_inv)
dev_query_pair['duplicate_qids'] = dev_query_pair.q_std.map(q_qid_dict_inv)
dev_query_pair.duplicate_qids = dev_query_pair.duplicate_qids.astype('str')
dev_query_pair = dev_query_pair.groupby(['q_sim', 'qid']).apply(lambda v: ','.join(v['duplicate_qids'])).reset_index(name='duplicate_qids')[['qid', 'q_sim', 'duplicate_qids']]
dev_query_pair.to_csv('F:/Projects/data/corpus/qa/FinanceFAQ/FinanceFAQ_dev.tsv', sep='\t', index=False)

### 阶段二数据集